In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")
df.head()

**Knowing my data**

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.drop(columns=['Unnamed: 32','id'],inplace=True)  #dropping the columns that are not required at all
df.columns

In [ ]:
df.shape

In [ ]:
df['diagnosis'].unique()

# What does the column name depict?

* Diagnosis (M = malignant, B = benign) 
* Radius (This is the mean distances from center to points on the perimeter) 
* Texture (This is the standard deviation of gray scale values) 
* Perimeter Area Smoothness (This is the local variation in radius lengths) 
* Compactness (This is the perimeter squared divided by the area and subtracted 1.0 from its value) (perimiter^2 / area - 10)
* Concavity (Severity of convace portions of the contour ) 
* Concave points (Number of concave portions of the contour ) 
* Symmetry Fractal dimension ("coastline approximation" -1)

*The mean, standard error and "worst" or largest which is actually the mean of the three largest values of these features were computed for each image, resulting in 30 features.* 

**This analysis aims to observe which features are most helpful in predicting malignant or benign cancer and to see general trends that may aid us in model selection and hyper parameter selection. The goal is to classify whether the breast cancer is benign or malignant.**

In [ ]:
# Describing the head of the loaded dataframe 
df.describe()

**Defining my X and Y values**

In [ ]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [ ]:
X.shape

In [ ]:
y.shape

**Data Analysis**

In [ ]:
# Benign Vs Malignant
value_count = df['diagnosis'].value_counts()  
plt.figure()

# Plotting the Count for the value counts in the diagnosis column
value_count.plot(kind = "bar", color = "blue", rot=0)
plt.ylabel("Counts")
plt.title("A bar chart showing the count of Benign Vs Malignant Labels")
plt.grid(True)
plt.show()

In [ ]:
# Displaying a violin plot for all the features 
labels = y 
input_features = X

# Normalizing the dataframe 
data = (input_features - input_features.mean()) / (input_features.std())
input_features = pd.concat([y, data.iloc[:,:]], axis = 1)

data = pd.melt(input_features, id_vars = "diagnosis", var_name = "features", 
              value_name = "value")

# Plotting the first Ten feature
plt.figure(figsize = (18, 7))
sns.violinplot(x = "features", y = "value", hue = "diagnosis", data = data, split = True, 
              inner = "quart")
plt.xticks(rotation = 90)
plt.grid(True)
plt.show()

In [ ]:
# Displaying a Correlation matrix of all the features of the breast cancer dataset.
f,ax = plt.subplots(figsize=(20, 18))
sns.heatmap(X.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
# Setting the figure of the Graph 
plt.figure(figsize=(18, 7))

# Plotting a scatter plot of diagnosis against radius_mean
plt.scatter(df['diagnosis'], df['radius_mean'])

# Displaying the graph 
plt.show()

In [ ]:
# Plotting a boxplot of the "Area_mean","area_se" and "area_worst"
boxplot = df.boxplot(column = ["area_mean","area_se","area_worst"], by="diagnosis", 
                    layout = (3, 1), figsize=(19, 9))
# Showing the boxplot
plt.show()

In [ ]:
# Plotting a boxplot of the means of different features
boxplot = df.boxplot(column = ["area_mean","perimeter_mean","compactness_mean"], by="diagnosis", 
                    layout = (3, 1), figsize=(19, 9))
# Showing the boxplot
plt.show()

In [ ]:
# Plotting a boxplot of the means of different features
boxplot = df.boxplot(column = ["concave points_mean","concavity_mean","smoothness_mean"], by="diagnosis", 
                    layout = (3, 1), figsize=(19, 9))
# Showing the boxplot
plt.show()

In [ ]:
# Plotting a boxplot of the means of different features
boxplot = df.boxplot(column = ["fractal_dimension_mean","radius_mean","symmetry_mean","texture_mean"], by="diagnosis", 
                    layout = (4, 1), figsize=(19, 9))
# Showing the boxplot
plt.show()

**Getting inside our model**

**SVM**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
# Creating scaled set to be used in model to improve the results
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Import Library of Support Vector Machine model
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Create a Support Vector Classifier
svc = svm.SVC()

# Hyperparameter Optimization
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

# Run the grid search
grid_obj = GridSearchCV(svc, parameters)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the svc to the best combination of parameters
svc = grid_obj.best_estimator_

# Train the model using the training sets 
svc.fit(X_train,y_train)


In [ ]:
# Prediction on test data
y_pred = svc.predict(X_test)

In [ ]:
# Calculating the accuracy
from sklearn.metrics import accuracy_score
acc_svm = round( accuracy_score(y_test, y_pred) * 100)
print( 'Accuracy of SVM model : ', acc_svm )

# The model is working at an accuracy of 97%

**This notebook is a part of task on SVM**